In [1]:
##Loading required package library

import os, glob, math, cv2, time
import numpy as np
from skimage import io
from joblib import Parallel, delayed
import cPickle as pickle
from skimage.transform import resize

In [13]:
## image reading and appending to a list
f =[]
for j in range(10):
    print('Load folder c{}'.format(j))
    path = os.path.join('/home/mohammad/Downloads/imgs/train', 'c' + str(j)+'/*')
    f.append(len(glob.glob(path)))

Load folder c0
Load folder c1
Load folder c2
Load folder c3
Load folder c4
Load folder c5
Load folder c6
Load folder c7
Load folder c8
Load folder c9


In [18]:
## getting image class name from folder name
c =[]
for j in range(10):
    c.append('c'+str(j))
#f = zip(c,f)
#c

['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']

In [22]:
## Data Exploration ##
## histogram plot to check distrbution
import matplotlib.pyplot as plt
#histogram=plt.figure()
f = np.asarray(f)
y_pos = np.arange(len(c))
plt.bar(y_pos,f, align='center', alpha=0.5)
plt.xticks(y_pos, c)
plt.ylabel('Total number of pictures')
plt.title('')
 
#plt.show()
plt.savefig('/home/mohammad/picture_hist.png',dpi=300)

In [3]:
### image normalization and resizing in parallel
### codes taken kaggle.com
img_size = 50
sz = (img_size, img_size)

nprocs = 3 ## number of processors

## image resize functions
def resize_image(img_file):
    img = cv2.imread(img_file,0)
    img = cv2.resize(img, sz).astype('float32') / 255.0
    return img

start = time.time()
X_train = []
Y_train = []

##loop image reading, and resizing using parallel processing
for j in range(10):
    print('Load folder c{}'.format(j))
    path = os.path.join('/home/mohammad/Downloads/imgs/train', 'c' + str(j), '*.jpg')
    files = glob.glob(path)
    X_train.extend(Parallel(n_jobs=nprocs)(delayed(resize_image)(im_file) for im_file in files))
    Y_train.extend([j]*len(files))
    
end = time.time() - start
print("Time: %.2f seconds" % end)



Load folder c0
Load folder c1
Time: 49.05 seconds


In [16]:

### image normalization and resizing in parallel for rgb
img_size = 50
sz = (img_size, img_size)

def process_image(img_file):
    img = cv2.imread(img_file)
    img = cv2.resize(img, sz).transpose((2,0,1)).astype('float32')
    img[:,:,0] -= 103.939
    img[:,:,1] -= 116.779
    img[:,:,2] -= 123.68
    return img


nprocs = 3
start = time.time()
X_train = []
Y_train = []

for j in range(10):
    print('Load folder c{}'.format(j))
    path = os.path.join('/home/mohammad/Downloads/imgs/train', 'c' + str(j), '*.jpg')
    files = glob.glob(path)
    X_train.extend(Parallel(n_jobs=nprocs)(delayed(process_image)(im_file) for im_file in files))
    Y_train.extend([j]*len(files))
    
end = time.time() - start
print("Time: %.2f seconds" % end)




Load folder c0
Load folder c1
Load folder c2
Load folder c3
Load folder c4
Load folder c5
Load folder c6
Load folder c7
Load folder c8
Load folder c9
Time: 186.84 seconds


In [17]:
##chekcing train and test shape
x= np.array(X_train)
y = np.array(Y_train)
print x.shape, y.shape

(22424, 3, 50, 50) (22424,)


In [18]:
##saving file
f = open('/home/mohammad/img_data_rgb.p', 'wb')   
pickle.dump(x, f)          
f.close()  

In [19]:
##saving file
f = open('/home/mohammad/img_label_rgb.p', 'wb')  
pickle.dump(y, f)          # dump data to f
f.close()  

In [10]:
## generating train, test and validation data set
from sklearn.model_selection import train_test_split
X_train1, X_test, y_train1, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train1, y_train1, test_size=0.33, random_state=42)

In [10]:
## for rgb
img_size = 224
sz = (img_size, img_size)

def process_image(img_file):
    img = io.imread(img_file)    
    img = resize(img, sz,mode='nearest').transpose((2,0,1)).astype('float32')
    img[:,:,0] -= 103.939
    img[:,:,1] -= 116.779
    img[:,:,2] -= 123.68
    return img


nprocs = 3
start = time.time()
X_train = []
Y_train = []

np.arange(6,8)

array([6, 7])

In [7]:
for j in np.arange(6,8):
    print('Load folder c{}'.format(j))
    path = os.path.join('/home/mohammad/Downloads/imgs/train', 'c' + str(j), '*.jpg')
    files = glob.glob(path)
    X_train.extend(Parallel(n_jobs=nprocs)(delayed(process_image)(im_file) for im_file in files))
    Y_train.extend([j]*len(files))
    
end = time.time() - start
print("Time: %.2f seconds" % end)


Load folder c6
Load folder c7
Time: 72.24 seconds


In [8]:
X_train= np.array(X_train)
#Y_train = np.array(Y_train)
#X_train=[]
#Y_train = []
print  X_train.shape

(4327, 3, 224, 224)


In [9]:
##saving data as h5 file
import sys
sys.path.append('/home/mohammad/anaconda/lib/python2.7/site-packages')
import h5py
with h5py.File('/home/mohammad/data3.h5', 'w') as hf:
    hf.create_dataset('dataset_1', data=X_train)

In [11]:
import sys
sys.path.append('/home/mohammad/anaconda/lib/python2.7/site-packages')
import h5py

with h5py.File('/home/mohammad/data1.h5','r') as hf:
    data = hf.get('dataset_1')
    data = np.array(data)
    
print data.shape

(4663, 3, 224, 224)


In [9]:
f = open('/home/mohammad/img_data_vgg_p1.p', 'wb')   
pickle.dump(x, f)          
f.close()  

MemoryError: 

In [ ]:
f = open('/home/mohammad/img_label_rgb.p', 'wb')  
pickle.dump(y, f)          # dump data to f
f.close() 